In [ ]:
import tensorflow as tf
from time import sleep
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing import image
import cv2
import numpy as np
from tensorflow.keras import models, layers, regularizers

input_shape = (128,128,3) # img_rows, img_colums, color_channels
num_classes = 7

Xception=tf.keras.applications.InceptionResNetV2(weights='imagenet',input_shape=input_shape, include_top=False)
# for layer in MobileNetV3.layers:
#     layer.trainable = False
# Build Model
model = models.Sequential()
# load the model
model.add(Xception)

model.add(layers.Flatten())
model.add(layers.Dense(1024, activation='relu'))
model.add(layers.Dropout(0.75))
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dropout(0.50))
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dropout(0.25))
model.add(layers.Dense(128,activation='relu'))
model.add(layers.Dropout(0.25))
model.add(layers.Dense(num_classes, activation='softmax'))

model.load_weights(r"C:\Users\aayus\Downloads\emotion_Classifi_ver.h5")


face_classifier = cv2.CascadeClassifier(r"C:\Users\aayus\OneDrive\Desktop\haarcascade_frontalface_default.xml")


emotion = ['ANGRY','DISGUST','FEAR','HAPPY','SAD', 'SURPRISE', 'NEUTRAL']

cap = cv2.VideoCapture(0)
while True:
    _, frame = cap.read()
    labels = []
    gray = frame
    faces = face_classifier.detectMultiScale(gray)

    for (x,y,w,h) in faces:
        cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,255),3)
        FaceGray = gray[y:y+h,x:x+w]
        FaceGray = cv2.resize(FaceGray,(128,128),interpolation=cv2.INTER_AREA)



        if np.sum([FaceGray])!=0:
            FaceGrayScaled = FaceGray.astype('float')/255.0
            FaceGrayScaled = img_to_array(FaceGrayScaled)
            FaceGrayScaled = np.expand_dims(FaceGrayScaled,axis=0)
            prediction =model.predict(FaceGrayScaled)[0]
            label=emotion[prediction.argmax()]
            label_position = (x,y)
            cv2.putText(frame,label,label_position,cv2.FONT_HERSHEY_COMPLEX,1,(0,255,0),2)
        else:
            cv2.putText(frame,'FACE NOT FOUND',(30,80),cv2.FONT_HERSHEY_COMPLEX,1,(0,255,0),2)
    cv2.imshow('EMOTION DETECTION',frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
